In [1]:
import icecube
from icecube.icetray.i3logging import log_info, log_fatal
from icecube import dataclasses, dataio, icetray, simclasses, MuonGun, hdfwriter
import matplotlib.pyplot as plt
import numpy as np
from I3Tray import I3Tray
import glob

In [2]:
#Function to read the GCD file and make the extruded polygon which
#defines the edge of the in-ice array
def MakeSurface(gcdName, padding):
    file = dataio.I3File(gcdName, "r")
    frame = file.pop_frame()
    while not "I3Geometry" in frame:
        frame = file.pop_frame()
    geometry = frame["I3Geometry"]
    xyList = []
    zmax = -1e100
    zmin = 1e100
    step = int(len(geometry.omgeo.keys())/10)
    print("Loading the DOM locations from the GCD file")
    for i, key in enumerate(geometry.omgeo.keys()):
        if i % step == 0:
            print( "{0}/{1} = {2}%".format(i,len(geometry.omgeo.keys()), int(round(i/len(geometry.omgeo.keys())*100))))
            
        if key.om in [61, 62, 63, 64] and key.string <= 81: #Remove IT...
            continue

        pos = geometry.omgeo[key].position

        if pos.z > 1500:
            continue
            
        xyList.append(pos)
        i+=1
    
    return MuonGun.ExtrudedPolygon(xyList, padding) 

def get_position_along_track(particle, length):
    return particle.pos + dataclasses.I3Position(length, particle.dir.theta, particle.dir.phi, dataclasses.I3Position.sph)


In [3]:
class MuonsInIce(icetray.I3Module):
    """
        This module is for atmospheric muon information at the detector surface
    """
    def __init__(self,ctx):
        icetray.I3Module.__init__(self,ctx)

        self.gcdFile = ""
        self.AddParameter("GCDFile", "GCD file which defines the in-ice volume", self.gcdFile)

        self.padding = 0. * icetray.I3Units.m # default no padding
        self.AddParameter("Padding", "", self.padding)

    def Configure(self):
        pass
            
    def DAQ(self, frame):
        
        tracklist = frame["MMCTrackList"]
        muon_energies_tracklist = [track.Ei for track in tracklist]
        # save to frame
        muon_map                      = dataclasses.I3MapStringDouble()
        muon_map["N"]                 = len(muon_energies_tracklist)
        muon_map["HighestMuonEnergyTrack"] = max(muon_energies_tracklist)
        muon_map["TotalEnergyTrack"]       = sum(muon_energies_tracklist)
        frame["MuonAtDetectorBoundary"] = muon_map
        self.PushFrame(frame)
        
        
    def Finish(self):
        pass

In [4]:
filelist = list(glob.glob("/data/sim/IceCube/2020/filtered/level2/CORSIKA-in-ice/20904/0198000-0198999/Level2*.i3.zst"),)
n_files = 30 # how many files to use?
filelist = filelist[0:n_files] 
print("Number of files ", len(filelist))
gcdfile = "/data/user/axelpo/LLP-at-IceCube/dark-leptonic-scalar-simulation/resources/GeoCalibDetectorStatus_2021.Run135903.T00S1.Pass2_V1b_Snow211115.i3.gz"

tray = I3Tray()

tray.Add("I3Reader", FileNameList=filelist)
tray.Add(MuonsInIce, GCDFile = gcdfile)
tray.Add(
    hdfwriter.I3SimHDFWriter,
    keys=["PolyplopiaPrimary", "CorsikaWeightMap", "MuonAtDetectorBoundary"],
    output="Muons_in_ice_"+str(n_files)+"files.hdf5",
)

tray.Execute()

Number of files  30


NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))
WARN (I3TableWriter): 2 SubEventStreams ['InIceSplit','NullSplit',] were seen but not booked because they were not passed as part of the 'SubEventStreams' parameter of I3TableWriter (which was configured as ['SimHDFWriter',]). To book events from these streams, add them to the 'SubEventStreams' parameter of I3TableWriter. (I3TableWriter.cxx:479 in void I3TableWriter::Finish())
